In [25]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime

# ticker_symbol_input = yf.Ticker("MSFT")
# print(ticker_symbol_input.info)

In [26]:
# parent/super class
class EDWARD_dashboard:
    def __init__(self, ticker_symbol,  contract_type, exercise_price, expiration_date):
        self.ticker_symbol = ticker_symbol
        self.current_date = datetime.now().strftime("%Y-%m-%d")
        self.contract_type = contract_type
        self.exercise_price = exercise_price
        self.expiration_date = expiration_date

    def days_between(self):
        d1, d2 = datetime.strptime(self.current_date, "%Y-%m-%d"), datetime.strptime(self.expiration_date, "%Y-%m-%d")
        diff_times = abs((d2 - d1).days)
        return diff_times


    def portfolio(self):
        ticker_symbol = yf.Ticker(self.ticker_symbol)
        pass

    def futures_formula(self, spot_price, nominal_rate, dividend_yield, inflation_rate):
        Price_contract =  spot_price * np.e**(((1+nominal_rate)/(1+inflation_rate))* (self.days_between()))

    def __str__(self):
        return f" current date {str(self.current_date)}; difference between dates {str(self.days_between())}"

print(EDWARD_dashboard(3,3,3,"2023-11-06"))

 current date 2022-11-06; difference between dates  365


In [28]:
# User Inputs
input_symbol = input("Ticker Symbol you would like to use? ")
# ticker_symbol = yf.Ticker(input_symbol)
input_contract_type = input("What contracct type would you like to use (Futures, Options, Swaps)? ")
input_exercise_price = int(input("What price would you like to exercise the shares at? "))
input_expiration_date = input("What date would you like to set the expiration? (Y-M-D)")
# EDWARD_dashboard(input_symbol, input_contract_type, input_exercise_price, input_expiration_date)


ValueError: invalid literal for int() with base 10: ''